In [1]:
import requests
import json
import pandas as pd
import numpy as np
from urllib.parse import urljoin
import time
import logging
import csv
from tqdm import tqdm
import glob
import json
import scipy
import pandas as pd
import numpy as np
from scipy.sparse import csr_matrix
from scipy import sparse
import pickle
import torch
from torch_geometric.data import Data
import torch.nn.functional as F
import torch_geometric.transforms as T
from torch_geometric.nn import SplineConv,GCNConv
from torch_geometric.data import InMemoryDataset, download_url

GRAPHSQL_URL = "http://192.168.20.241:9000/query/OneMonthNet/"

############################ util functions ############################
def query_graphsql_ori(query_name, para_string):
    remaining_url = "{}?{}".format(query_name, para_string)
    url=urljoin(GRAPHSQL_URL, remaining_url)
    print('query:{}\nparas:{}\nrequest for url: {}'.format(query_name, para_string.split('&'), url))
    print('---------------------------------------------------------------------------------------------------')
    result = requests.get(url)
    result_json = json.loads(result.text)
    return result_json


def query_graphsql(query_name, para_string):
    remaining_url = "{}?{}".format(query_name, para_string)
    url=urljoin(GRAPHSQL_URL, remaining_url)
    st = time.time()
    print('query:{}\nparas:{}\nrequest for url: {}'.format(query_name, para_string.split('&'), url))
    print('---------------------------------------------------------------------------------------------------')
    try:
        result = requests.get(url)
        result_json = json.loads(result.text)
        if result_json['error']:
            logging.error(result_json['message'])
            print('run query failed')
            return None
        print('run query finish, use {} seconds\n\n'.format(time.time() - st))
        return result_json
    except Exception as e:
        print('failed')
        
        
def _update_max_min_date(node_name):
    '''
    get the max & min date for the given node type based on its history update dates;
    para: node_name: 'Device'; check all the node_name type in gsql graph schema: OneMonthNet;
    '''
    query_name = 'update_max_min_date'
    paras = 'node={}'.format(node_name)
    return query_graphsql(query_name, paras)


def _node_cutoff_filter(start_time, end_time, node_type):
    '''
    note that call _update_max_min_date before run this query;
    select those nodes with given node_type exist between start_time and end_time;
    paras:start_time:'2019-06-01 18:42:22'
    paras:end_time:'2019-07-01 18:42:22'
    paras:node_type:'User'; check all the node_name type in gsql graph schema: OneMonthNet;
    '''
    query_name = 'node_cutoff_filter'
    paras = 'start_t={}&end_t={}&node={}'.format(start_time, end_time, node_type) 
    return query_graphsql(query_name, paras)
    
    
########################################################################

# def label_get(start_date):
#     'old version and has been deprecated now, pls use label_get2 query;'
#     query_name = 'label_get'
#     paras = 'start_date={}'.format(start_date) 
#     return query_graphsql(query_name, paras)


def label_get2(start_date, end_date, loanstyle):
    '''
    get loans with the given loanstyle whose fundtime between start_date & end_date;
    paras: start_date: '2019-06-01 00:00:00'
    paras: end_date: '2019-07-01 00:00:00'
    paras: loanstyle: '绿卡30天1期'
    '''
    query_name = 'label_get2'
    paras = 'start_date={}&end_date={}&loan_type={}'.format(start_date, end_date, loanstyle) 
    return query_graphsql(query_name, paras)['results'][0]['loanlabelSHOW']


#1
def reset(node):
    '''
    '''
    query_name = 'reset'
    paras = 'node={}'.format(node) 
    return query_graphsql(query_name, paras)


#2
def pageRank_train(start_t, end_t, node, maxChange, maxIter, damping, query_name):
    '''
    '''
    paras = 'start_t={}&end_t={}&node={}&maxChange={}&maxIter={}&damping={}'.format(start_t, end_t, node, maxChange,maxIter,damping) 
    return query_graphsql(query_name, paras)


#3
def pageRank_appr_files(file_abs_path, query_name):
    '''
    draft 1 for pageRank in cashbus graph; 
    Only works for user-device relationship(single edge type & double vertex type);
    Only output nodes(type: user or device) with pg_score !=0;
    '''
    paras = 'file_path={}'.format(file_abs_path) 
    return query_graphsql(query_name, paras) 


def pyG_prepare_label(st, et, node):
    query_name = 'get_default_now'
    paras = 'start_t={}&end_t={}&node={}'.format(st, et, node)
    return query_graphsql(query_name, paras) 

    
def pyG_prepare_edge(st, et, node):
    query_name = 'pyG_pre'
    paras = 'start_t={}&end_t={}&node={}'.format(st,et,node)
    return query_graphsql(query_name, paras) 


def pyG_prepare_feat(st, et, node):
    query_name = 'pyG_pre_ori'
    paras = 'start_t={}&end_t={}&node={}'.format(st,et,node)
    return query_graphsql(query_name, paras) 


#2
def connected_comp_train(start_t, end_t, node):
    '''
    '''
    query_name='conn_comp'
    paras = 'start_t={}&end_t={}&node={}'.format(start_t, end_t, node) 
    return query_graphsql(query_name, paras)


#3
def connected_comp_appr_files(file_abs_path):
    '''
    draft 1 for pageRank in cashbus graph; 
    Only works for user-device relationship(single edge type & double vertex type);
    Only output nodes(type: user or device) with pg_score != 0;
    '''
    paras = 'file_path={}'.format(file_abs_path)
    query_name = 'conn_comp_check'
    return query_graphsql(query_name, paras)


def train(st, et, node_type, reset_bool):
    """
    paras: st: start time of a period, only node within the restriction will be trained and given a pgscore;
    paras: et: end time of the period;
    paras: node_type: determine which nodetype will be trained;
    """
    assert node_type in ["Device", "PhoneNumber"]
    if reset_bool:
        reset(node_type)
    pageRank_train(st, 
                   et, 
                   node_type, 
                   10, 3, 0.6,
                   "pageRank_train_{}".format(node_type.lower()))
    return None


def test_with_local_files(nodetype, file_path):
    """
    paras: node_type: determine which nodetype will be test;
    paras: file_path: only provide a path to save test nodes id;
    """
    test_res2 = pageRank_appr_files(file_path, 
                                    "pageRank_appr_files_{}".format(nodetype.lower()))
    user_pg_res = [i['attributes'] for i in test_res2['results'][0]['test_set']]
    user_pg_df2 = pd.DataFrame.from_dict(user_pg_res)
    user_pg_df2.rename(columns={'prim_id': 'username'}, inplace=True)
    return test_res2, user_pg_df2


def check_phone(phone):
    paras = 'm1={}'.format(phone)
    query_name = 'check_phone'
    return query_graphsql(query_name, paras)


def get_edge(res):
    edge = []
    for dic in res['results'][0]['vv']:
        b = dic['v_id']
        a = dic['attributes']['vv.@node_all_neighs']
        bb = len(a)*[b]
        temp_edge = list(zip(a, bb))
        edge.extend(temp_edge)
    return edge


def get_feat(res_ori):
    feat_DIC = {}
    for dic in res_ori['results'][0]['vv']:
        feat_dic = dic['attributes']['vv.@node_date_calls']
        feat = []
        v_id = dic['v_id']
        for date in ['20190601', '20190602', '20190603', '20190604']:
            if date not in feat_dic:
                feat.append(0)
            else:
                feat.append(len(feat_dic[date]))
        feat_DIC[v_id] = feat
    return feat_DIC


def get_label(res_label):
    DIC = {}
    for dic in res_label['results'][0]['vv']:
        label = dic['attributes']['vv.@node_all_labels']
        DIC[dic['v_id']]=label
    return DIC
    

# 1. Prepare & save to local 

In [ ]:
label1 = pd.read_csv('/data-0/gsm/qb_one_month/loan_label_1_1901_1907.csv')
label2 = pd.read_csv('/data-0/gsm/qb_one_month/loan_label_2_new_1901_1907.csv')
label2.columns = list(label1.columns)+['nouse']

# xiaomajie 给的表；
loan_detail_08_09_df = pd.read_csv('/data-0/qibo/Gdata/oneMonth/qb_temp_loan_detail_08_09.csv')
loan_detail_08_09_df.username = loan_detail_08_09_df.username.astype(str)

def test_loan_info(loanstyle, st, et):
    # 给定两个时间范围，给出所在时间内的 funded loan 信息；
    temp1 = label1[(label1.funddate<et)&(label1.funddate>=st)]
    temp2 = label2[(label2.funddate<et)&(label2.funddate>=st)]
    final = temp1.append(temp2)
    if loanstyle == 'all':
        return final
    return final[final.loanstyle == loanstyle]

def source_table(loanstyle, st, et):
    #xiaomajie biao;
    test_funded_loans = loan_detail_08_09_df[(loan_detail_08_09_df.fundtime>=st)&(loan_detail_08_09_df.fundtime<et)]
    #label 表;
    test_label = test_loan_info(loanstyle, st, et)
    #merge;
    table3 = pd.merge(test_funded_loans, test_label, on='loanid')
    return table3

test_st='2019-06-01 00:00:00'
test_et='2019-06-05 00:00:00'

source_df = source_table('绿卡30天1期', test_st, test_et)
four_days_label = source_df[['username', 'fundtime', 'default_now']]

root ='/home/qibo/all_project/Graph反欺诈/PYG/raw'
four_days_label.to_csv(root+'/four_days_label.csv', index=False)

In [2]:
test_st='20190601'
test_et='20190604'

res_edge = pyG_prepare_edge(test_st, test_et, "PhoneNumber")
res_feat = pyG_prepare_feat(test_st, test_et, "PhoneNumber")
# res_label = pyG_prepare_label(test_st, test_et, "PhoneNumber")


query:pyG_pre
paras:['start_t=20190601', 'end_t=20190604', 'node=PhoneNumber']
request for url: http://192.168.20.241:9000/query/OneMonthNet/pyG_pre?start_t=20190601&end_t=20190604&node=PhoneNumber
---------------------------------------------------------------------------------------------------
run query finish, use 94.4853093624115 seconds


query:pyG_pre_ori
paras:['start_t=20190601', 'end_t=20190604', 'node=PhoneNumber']
request for url: http://192.168.20.241:9000/query/OneMonthNet/pyG_pre_ori?start_t=20190601&end_t=20190604&node=PhoneNumber
---------------------------------------------------------------------------------------------------
run query finish, use 115.0154185295105 seconds


query:get_default_now
paras:['start_t=20190601', 'end_t=20190604', 'node=PhoneNumber']
request for url: http://192.168.20.241:9000/query/OneMonthNet/get_default_now?start_t=20190601&end_t=20190604&node=PhoneNumber
-----------------------------------------------------------------------------------

In [4]:
feat_DIC = get_feat(res_feat)
edge = get_edge(res_edge)

source = set(feat_DIC.keys())
a1 = np.array(edge).transpose()[0]
a2 = np.array(edge).transpose()[1]
edge_set = set(a1).union(set(a2))

# 2. load and process

In [20]:
with open(root+'raw/feat.json') as json_file:
    x = json.load(json_file)

In [25]:
pd.DataFrame.from_dict(x)

,07962104310,077195232611,07962133824,077195202442,089895147706,07902106988,077195241135,07992089059,087168284381,09513842096,...,15260014761,15260138012,13656077398,15265355431,15261115158,15107020440,15116227437,13792499758,13780678904,13789218135
0,7,0,4,1,1,0,0,1,14,13,...,1,1,1,1,1,1,1,1,1,1
1,4,0,9,0,2,1,0,0,12,10,...,0,0,0,0,0,0,0,0,0,0
2,4,2,1,1,1,1,0,0,20,10,...,0,0,0,0,0,0,0,0,0,0
3,4,0,7,1,1,1,1,3,15,9,...,0,0,0,0,0,0,0,0,0,0


In [47]:
# root ='/home/qibo/all_project/Graph反欺诈/PYG/raw'

# with open(root+'/feat.json') as json_file:
#     x = json.load(json_file)
    
# with open(root+'/x_phone2idx.json') as json_file:
#     x_phone2idx = json.load(json_file)

# with open(root+'/edge.json') as json_file:
#     edge = json.load(json_file)

# root ='/home/qibo/all_project/Graph反欺诈/PYG/raw/'
# labels = pd.read_csv(root+'four_days_label.csv')


def __get_mask_idx(labels_df, train_valid_split=0.8):
    train_idx = labels_df[labels_df.fundtime<'2019-06-03 00:00:00'].username.tolist()
    test_idx = labels_df[labels_df.fundtime>='2019-06-04 00:00:00'].username.tolist()
    split_idx = int(len(train_idx)*train_valid_split)
    train = train_idx[:split_idx]
    valid = train_idx[split_idx:]
    print(len(train), len(valid), len(test_idx))
    return train, valid, test_idx

def _get_final_label(labels, new_x_phone2idx):
    
    label_dict = labels[['username', 'default_now']].set_index('username').to_dict()['default_now']
    train_phone, valid_phone, test_phone = __get_mask_idx(labels)
    fake_labels = np.zeros(len(new_x_phone2idx))-1

    for phone in label_dict:
        idx = new_x_phone2idx[str(phone)]
        fake_labels[idx] = label_dict[int(phone)]   

    train_mask = [new_x_phone2idx[str(phone)] for phone in train_phone]
    fake_train_labels = np.zeros(len(new_x_phone2idx))
    fake_train_labels[train_mask] = 1

    valid_mask = [new_x_phone2idx[str(phone)] for phone in valid_phone]
    fake_valid_labels = np.zeros(len(new_x_phone2idx))
    fake_valid_labels[valid_mask] = 1
    
    test_mask = [new_x_phone2idx[str(phone)] for phone in test_phone]
    fake_test_labels = np.zeros(len(new_x_phone2idx))
    fake_test_labels[test_mask] = 1
    
    return fake_labels, fake_train_labels, fake_valid_labels, fake_test_labels


def _add_new_feat(x, test_df):
    not_in_edge_lis = []
    for i in test_df.username:
        if str(i) not in x:
            x[str(i)] = [0, 0, 0, 0]
            not_in_edge_lis.append(str(i))
    return x, not_in_edge_lis


def _add_new_edge(edge, test_df, not_in_edge_lis):
    for i in test_df.username:
        if str(i) in not_in_edge_lis:
            edge.append([str(i), str(i)])
    return edge
    
def _add_phone2ix(x_phone2idx, not_in_edge_lis):
    old_len = len(x_phone2idx)
    i=0
    for new_phone in not_in_edge_lis:
        x_phone2idx[new_phone] = old_len+i
        i+=1
    return x_phone2idx

def get_all_info(x, edge, x_phone2idx, test_df):
    print(len(x), len(edge), len(x_phone2idx))
    print('--------------------------')
    x, not_in_edge_lis = _add_new_feat(x, test_df)
    edge = _add_new_edge(edge, test_df, not_in_edge_lis)
    x_phone2idx = _add_phone2ix(x_phone2idx, not_in_edge_lis)
    print(len(x), len(edge), len(x_phone2idx))
    fake_labels, train_mask, valid_mask, test_mask = _get_final_label(test_df, x_phone2idx)
    return x, edge, x_phone2idx, fake_labels, train_mask, valid_mask, test_mask
    

def read_cashbus_data(root):
    
    ##################### finish add test infos into the graph ############################
    with open(root+'/feat.json') as json_file:
        x = json.load(json_file)
    with open(root+'/x_phone2idx.json') as json_file:
        x_phone2idx = json.load(json_file)
    with open(root+'/edge.json') as json_file:
        edge = json.load(json_file)
    labels = pd.read_csv(root+'/four_days_label.csv')
    x, edge, x_phone2idx, y, train_mask, valid_mask, test_mask = get_all_info(x, edge, x_phone2idx, labels)  
    
    ##################### finish add test infos into the graph ############################
    
    feat_mat = []
    for k,v in tqdm(x.items()):
        feat_mat.append(v)
    x = torch.tensor(feat_mat, dtype =torch.float)    
    y = torch.tensor(y).squeeze()
    edge = np.array(edge).T
    row1 = [x_phone2idx[str(i)] for i in edge[0]]
    row2 = [x_phone2idx[str(i)] for i in edge[1]]
    new_edges = torch.tensor(np.stack([row1, row2]))
    
    ##################### finish all ############################
    data = Data(x=x, edge_index=new_edges, y=y)
    data.train_mask = torch.tensor(train_mask, dtype=torch.uint8)
    data.val_mask = torch.tensor(valid_mask, dtype=torch.uint8)
    data.test_mask = torch.tensor(test_mask, dtype=torch.uint8)
    return data

class CashBus(InMemoryDataset):
    def __init__(self, root, transform=None, pre_transform=None):
        super(CashBus, self).__init__(root, transform, pre_transform)
        print('processed_path:{}'.format(self.processed_paths))
        self.data, self.slices = torch.load(self.processed_paths[0])

    @property
    def raw_file_names(self):
        return ['feat.json', 'x_phone2idx.json', 'edge.json', 'four_days_label.csv']

    @property
    def processed_file_names(self):
        return 'data8.pt'
    
    def download(self):
        pass

    def process(self):
        print('go pl, raw_dir:{}'.format(self.raw_dir))
        data = read_cashbus_data(self.raw_dir)
        data = data if self.pre_transform is None else self.pre_transform(data)
        data, slices = self.collate([data])
        torch.save((data, slices), self.processed_paths[0])

    def __repr__(self):
        return '{}()'.format(self.name)

In [50]:
root = '/home/qibo/all_project/Graph反欺诈/PYG/'
dataset = CashBus(root)
data = dataset[0]


processed_path:['/home/qibo/all_project/Graph反欺诈/PYG/processed/data8.pt']


In [ ]:
ssh-rsa AAAAB3NzaC1yc2EAAAADAQABAAACAQC0DUMLKlzc/kR/oJcgUWDVhuAu0KIJC1LmZOV6coAZhABu7R7G71my0LdDdLOPpwojyybGI7VyaUw4pdhjaleOYs0p4m6tHAkIb862BfJ8d8MStknZQPGr2i0bOsu6pDyxLB4KlxhID1aHJz1UhlJ5If2cetDKDe+CSf43zAPAepK86yHTtkZIAlMMNkM1oTv9Ly511vO0aoQaIW7oWKdyXHmIIuO0IleQOKhwPYjSLDQGozfyLVI+FCgjK9EOTI6K8NYsqVVuY3CTGNr/EzCeof/Ep3uv7a8RMQqgnn7CzELfie1YX/0pwmsPimMwgUwR4vrMrCGoFcptxAR/h+4Qe/RkqDr1W5dfH+Qx5nqDD0MfHZEL6M6WlKFfQp893tSzU1THNj0eDNqRVv97Jo/iOuEYO9az2jnWq59NoelNb32eIFaxpsHDVY5IOGW96D8DF+tj6A3ZzprpCxUAlmHoxoVYwVRasW6zq7BC7zMzhs5Kw6pfTIfWS3HtOeHkT+wd0a842N7MugKpViu72fwP6JcyeXVztJTzEFv6N+sivvoRT1QbGUUz08oxEZ+SmFbrKYbKvdTxInZB+zDl3+nftnlP3XzAooGtug8CpcEkDUiYJcrt1NJej1vw2NtVAJjCJUoVy0xURqdb9EbOGjHMNEUsHJ0jykDmdNw7H6x+HQ== qibo_2018@outlook.com


In [ ]:
ssh-rsa AAAAB3NzaC1yc2EAAAADAQABAAACAQDO53rheAywg+H/m5Ob+/5QKgiAJ4DQMADqCgInoL3cmJGLs8bweCDEugBdHz/eM8itwL9yhilnM0oTEHzn/WiA/Jw3ZvdANcu0u1q4JBwQx4/mfJ9jpwj0H9s5EPWmkfGt5mfWYa41zBqTvC/muIi1waTBzBTH95c/RS0hTMKt2OUOieVbBBdAqGDKQSTd3BHoxR74/cyTZ3H+iOBtsrxzOM4WC7iMVEgImRs8ffsqhxl4k0KXCJXxyOUxqgZDK5ZoXYmgyH4+pQEiLv9pNP97Aqo5hTGFIvglypMeBLp/cg699T5naVhzCZ0B/wjVt3Qy3Xxvtt/X4qs9nUhOqlkNMfWAtzKhLtfiOIOlINnq4W9ftCaA63HCJxFbpVPqyb8lCZM9xXQcnWOds2rIabmBuU7OOwLy8aO9U+21AJgWzKCiHhRPmsiXmJTE+nBX+57ChoxYJBGZhMjIVk5NypfOA3vPWraSWt+/q8MXn4Wkg241PyoLaUVAlP647n/xJu16Ukr2QVD5Wi2Ni1Q7W0zuUX8Z1rahwqK4kL5WyrI3l2go8j40+cHaAr6hPImzeGAo02lZvNUUR00YqrnThgMa7qwU8KPcmvn32ke+NG7oF+fCOf7UVRse3TPjbeNJtoeHQ/K8sWL+8HqdMyPLODnP4HZ/xJsta7gh/npP9L5Q9w== qibo_2018@outlook.com
